In [ ]:
dm._generate_kernels()


xmin = args.xmin
xmax = args.xmax
ymin = args.ymin
ymax = args.ymax

def convolve_per_proc(i):

    pf = fftconvolve(dm.image[ymin-dm.max_kern_size : ymax+1+dm.max_kern_size, 
                              xmin-dm.max_kern_size : xmax+1+dm.max_kern_size], 
                     dm.kernels[i], 'same').astype(np.half)

    sys.stderr.write('filtered kernel %d\n'%i)

    return pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size].flatten()

t = time.time()

mask = dm.mask[ymin:ymax+1, xmin:xmax+1]

perc = np.count_nonzero(mask) / float(mask.size)
sys.stderr.write('masked pixels = %.2f, ' % perc)

if perc == 0:
    sys.exit(0)

# sys.stderr.write('gabor filtering test ...')

# f = convolve_per_proc(88)

# sys.stderr.write('done in %f seconds\n' % (time.time() - t))

# t = time.time()

sys.stderr.write('gabor filtering ...')

# features = np.empty((dm.n_kernel, (ymax+1-ymin)*(xmax+1-xmin)), np.half)
# for i in range(dm.n_kernel):
#     features[i] = convolve_per_proc(i)
# features = features.T

features = Parallel(n_jobs=16, backend='threading')(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = Parallel(n_jobs=16)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = Parallel(n_jobs=4)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = Parallel(n_jobs=10)(delayed(convolve_per_proc)(i) for i in range(88,98))
# features = Parallel(n_jobs=16, backend='threading')(delayed(convolve_per_proc)(i) for i in range(88,96)+range(88,96))

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

# sys.exit(0)

t = time.time()
sys.stderr.write('transpose ...')

features = np.asarray(features).T  # n x 99

sys.stderr.write('done in %f seconds\n' % (time.time() - t))
